## **Collegamento Google Drive**

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **Valutazione Modelli**

## **Import librerie**

In [3]:
#utils
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import matplotlib.pyplot as plt

#modelli
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize

## *Install Optuna*

In [4]:
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.6 MB/s eta 0:00:00


# **PROTOCOLLO SUBJECT BIASED** (split randomico 70/30)

In [6]:
# Carico il dataset

#import dati combinati
data_combined = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/combinated_data.csv')
data_combined2 = data_combined.drop(['Key'], axis=1)

#elimino i valori NAN
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
dati_imputed = pd.DataFrame(imputer.fit_transform(data_combined2), columns=data_combined2.columns)

#normalizzazione dei dati
normalised_data = pd.DataFrame(normalize(dati_imputed, axis = 0))

# Caricamento delle etichette da un file CSV
etichette_df = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/etichette.csv')

# Estrazione delle etichette dal dataframe
etichette = etichette_df.iloc[:, 0].values

# Divisione dei dati in set di addestramento e test randomico secondo la regola 70/30
X_train, X_test, y_train, y_test = train_test_split(normalised_data, etichette, test_size=0.2, random_state=42)

# **Definizione funzioni obiettivo per l'ottimizzazione di Optuna** 

# **XGBoost**

In [7]:
# Verifica se la GPU è disponibile
USE_GPU = xgb.__version__.endswith('gpu')

# Definisci la funzione obiettivo per l'ottimizzazione di Optuna
def xgboost_objective(trial):
    # Parametri da ottimizzare per XGBoost

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'max_depth': trial.suggest_int('max_depth', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.00001, 0.1, log=True),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'tree_method': 'gpu_hist' if USE_GPU else 'auto',
        'n_jobs': -1,
        'verbosity': 0,
    }


    # Creazione del classificatore XGBoost con i parametri ottimizzati
    xgb_classifier = xgb.XGBClassifier(**params)

    # Addestramento del modello
    xgb_classifier.fit(X_train, y_train)

    # Valutazione sul set di test
    y_pred = xgb_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy

# **SVM**

In [8]:
# Definisci la funzione obiettivo per l'ottimizzazione di Optuna per SVM
def svm_objective(trial):
    # Definizione degli iperparametri da ottimizzare
    params = {
        'C': trial.suggest_float('C', 1e-6, 1e+6, log=True),
        'kernel': trial.suggest_categorical('kernel', ['sigmoid']),
        'degree': trial.suggest_int('degree', 1, 2),
        'gamma': trial.suggest_float('gamma', 1e-6, 1e+1, log=True),
        'coef0': trial.suggest_float('coef0', -1.0, 1.0),
    }


    # Creazione del classificatore SVM con i parametri ottimizzati
    svm_classifier = SVC(**params)

    # Addestramento del modello
    svm_classifier.fit(X_train,  np.ravel(y_train))

    # Valutazione sul set di test
    y_pred = svm_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy

# **Random Forest**

In [ ]:
# Definisci la funzione obiettivo per l'ottimizzazione di Optuna per Random Forest
def rf_objective(trial):
    # Parametri da ottimizzare per Random Forest
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'max_depth': trial.suggest_int('max_depth', 10, 300),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
    }

    # Creazione del classificatore Random Forest con i parametri ottimizzati
    rf_classifier = RandomForestClassifier(**params)

    # Addestramento del modello
    rf_classifier.fit(X_train,  np.ravel(y_train))
    # Valutazione sul set di test
    y_pred = rf_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy

# **Logistic Regression**

In [ ]:
# Definisci la funzione obiettivo per l'ottimizzazione di Optuna per Logistic Regression
def lr_objective(trial):
    # Parametri da ottimizzare per Logistic Regression
    params = {
        'C': trial.suggest_loguniform('C', 0.001, 100),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'saga']),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'max_iter': trial.suggest_int('max_iter', 100, 1000, step=100),
    }

    # Creazione del classificatore Logistic Regression con i parametri ottimizzati
    lr_classifier = LogisticRegression(**params)

    # Addestramento del modello
    lr_classifier.fit(X_train, np.ravel(y_train))

    # Valutazione sul set di test
    y_pred = lr_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy


# **Naive Bayes**

In [ ]:
# Definisci la funzione obiettivo per l'ottimizzazione di Optuna per Naive Bayes
def nb_objective(trial):
    # Parametri da ottimizzare per Naive Bayes
    params = {
        'var_smoothing': trial.suggest_loguniform('var_smoothing', 1e-10, 1e-1),
    }

    # Creazione del classificatore Naive Bayes con i parametri ottimizzati
    nb_classifier = GaussianNB(**params)

    # Addestramento del modello
    nb_classifier.fit(X_train, np.ravel(y_train))

    # Valutazione sul set di test
    y_pred = nb_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy


# **AdaBoost**

In [ ]:
# Definisci la funzione obiettivo per l'ottimizzazione di Optuna
def adaboost_objective(trial):
    # Parametri da ottimizzare per AdaBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'algorithm': trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R']),
        # Aggiungi altri parametri se necessario
    }

    # Creazione del classificatore AdaBoost con i parametri ottimizzati
    adaboost_classifier = AdaBoostClassifier(**params)

    # Addestramento del modello
    adaboost_classifier.fit(X_train, y_train)

    # Valutazione sul set di test
    y_pred = adaboost_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Salva i risultati dell'esecuzione corrente
    trial.set_user_attr("params", params)
    trial.set_user_attr("accuracy", accuracy)

    return accuracy

# **SEARCH BEST MODEL**
Effettuiamo 50 iterazioni per ottenere il miglior modello (n_trials=50)

## **XGBoost**

In [ ]:
# Creazione dello studio Optuna per XGBoost
xgboost_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri per XGBoost
xgboost_study.optimize(xgboost_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = xgboost_study.best_trial

# Ottenere i migliori parametri per il modello XGBoost
best_params = best_trial.params

# Creazione del modello XGBoost con i migliori parametri
best_xgboost_model = xgb.XGBClassifier(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_xgboost_model.fit(X_train, y_train)

# Salvare il miglior modello ottenuto per XGBoost
joblib.dump(best_xgboost_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")

# Salvataggio dei risultati in un DataFrame per XGBoost
xgboost_results = pd.DataFrame(xgboost_study.trials)
xgboost_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/xgboost_optuna_results.csv", index=False)

## **SVM**

In [ ]:
# Creazione dello studio Optuna per SVM
svm_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri per SVM
svm_study.optimize(svm_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = svm_study.best_trial

# Ottenere i migliori parametri per il modello SVM
best_params = best_trial.params

# Creazione del modello SVM con i migliori parametri
best_svm_model = SVC(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_svm_model.fit(X_train, np.ravel(y_train))

# Salvare il miglior modello ottenuto per SVM
joblib.dump(best_svm_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")

# Salvataggio dei risultati in un DataFrame
svm_results = pd.DataFrame(svm_study.trials)
svm_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/svm_optuna_results.csv", index=False)

## **Random Forest**

In [ ]:
# Creazione dello studio Optuna per Random Forest
rf_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri
rf_study.optimize(rf_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = rf_study.best_trial

# Ottenere i migliori parametri per il modello Random Forest
best_params = best_trial.params

# Creazione del modello Random Forest con i migliori parametri
best_rf_model = RandomForestClassifier(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_rf_model.fit(X_train, np.ravel(y_train))

# Salvare il miglior modello ottenuto per Random Forest
joblib.dump(best_rf_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")

# Salvataggio dei risultati in un DataFrame
rf_results = pd.DataFrame(rf_study.trials)
rf_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/rf_optuna_results.csv", index=False)

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Creazione dello studio Optuna per Logistic Regression
lr_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri per Logistic Regression
lr_study.optimize(lr_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = lr_study.best_trial

# Ottenere i migliori parametri per il modello Logistic Regression
best_params = best_trial.params

# Creazione del modello Logistic Regression con i migliori parametri
best_lr_model = LogisticRegression(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_lr_model.fit(X_train, np.ravel(y_train))

# Salvare il miglior modello ottenuto per Logistic Regression
joblib.dump(best_lr_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")

# Salvataggio dei risultati in un DataFrame
lr_results = pd.DataFrame(lr_study.trials)
lr_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/lr_optuna_results.csv", index=False)


# **Naive Bayes Classifier**

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Creazione dello studio Optuna per Naive Bayes Classifier
nb_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri per Naive Bayes Classifier
nb_study.optimize(nb_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = nb_study.best_trial

# Ottenere i migliori parametri per il modello Naive Bayes Classifier
best_params = best_trial.params

# Creazione del modello Naive Bayes Classifier con i migliori parametri
best_nb_model = GaussianNB(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_nb_model.fit(X_train, y_train)

# Salvare il miglior modello ottenuto per Naive Bayes Classifier
joblib.dump(best_nb_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")

# Salvataggio dei risultati in un DataFrame
nb_results = pd.DataFrame(nb_study.trials)
nb_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/nb_optuna_results.csv", index=False)


# **AdaBoost**

In [ ]:
# Creazione dello studio Optuna per AdaBoost
adaboost_study = optuna.create_study(direction='maximize')

# Esecuzione dell'ottimizzazione dei parametri per AdaBoost
adaboost_study.optimize(adaboost_objective, n_trials=50)

# Dopo aver eseguito l'ottimizzazione dei parametri
best_trial = adaboost_study.best_trial

# Ottenere i migliori parametri per il modello AdaBoost
best_params = best_trial.params

# Creazione del modello AdaBoost con i migliori parametri
best_adaboost_model = AdaBoostClassifier(**best_params)

# Addestramento del modello con tutti i dati di addestramento
best_adaboost_model.fit(X_train, y_train)

# Salvare il miglior modello ottenuto per AdaBoost
joblib.dump(best_adaboost_model, "/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")

# Salvataggio dei risultati in un DataFrame per AdaBoost
adaboost_results = pd.DataFrame(adaboost_study.trials)
adaboost_results.to_csv("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/adaboost_optuna_results.csv", index=False)


# **EVALUATE MODEL**

In [9]:
def evaluate_model(X_test, y_test, model_name, model):
    
    y_pred = model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy = accuracy_score(y_test, y_pred)
    # Calcolare la precisione, il richiamo e l'F1-score
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    cm = confusion_matrix(y_test, y_pred)
        
    # Crea un DataFrame con i risultati
    results = pd.DataFrame({'Model': [model_name],
                            'Accuracy': [accuracy],
                            'Precision': [precision],
                            'Recall': [recall],
                            'F1-score': [f1]})
    
    # Salva i risultati in un file CSV
    results.to_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Metriche_modelli/'+model_name+'_metrics_results.csv', index=False)

    # Calcolare la matrice di confusione normalizzata
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Creare una figura per la matrice di confusione
    plt.figure(figsize=(8, 6))

    # Plotting della matrice di confusione normalizzata utilizzando una heatmap
    sns.heatmap(cm_normalized, annot=True, cmap=plt.cm.Blues)

    # Impostazione dei titoli degli assi
    plt.title(model_name + ' Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Salva la matrice di confusione come un file PDF
    plt.savefig('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Matrici_di_confusione/' + model_name + '_confusion_matrix.pdf', format='pdf', bbox_inches='tight')

    # Chiude la figura
    plt.close()


# **MAIN**

In [ ]:
#XGBOOST
#Importa il modello migliore 
best_xgboost_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")
#Addestramento dei con tutti i dati di addestramento
best_xgboost_model.fit(X_train, np.ravel(y_train))
# Salva le metriche di valutazione del modello migliore di XGBoost
evaluate_model(X_test, y_test, "XGBoost",best_xgboost_model)
xgboost_pred = best_xgboost_model.predict(X_test)
#Calcola le metriche per la classificazione
accuracy_xgboost = accuracy_score(y_test, xgboost_pred)

#SVM
#Importa il modello migliore 
best_svm_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")
# Addestramento dei con tutti i dati di addestramento
best_svm_model.fit(X_train,np.ravel(y_train))
# Salva le metriche di valutazione del modello migliore di SVM
evaluate_model(X_test, y_test, "SVM", best_svm_model)
svm_pred= best_svm_model.predict(X_test)
# Calcola le metriche per la classificazione
accuracy_svm = accuracy_score(y_test, svm_pred)

#Random Forest
# Importa il modello migliore 
best_rf_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")
# Addestramento dei con tutti i dati di addestramento
best_rf_model.fit(X_train, np.ravel(y_train))
# Salva le metriche di valutazione del modello migliore di Random Forest
evaluate_model(X_test, y_test, "RF", best_rf_model)
rf_pred=best_rf_model.predict(X_test)
# Calcola le metriche per la classificazione
accuracy_rf = accuracy_score(y_test, rf_pred)

#Logistic Regression
#Importa il modello migliore 
best_lr_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")
# Addestramento dei con tutti i dati di addestramento
best_lr_model.fit(X_train,np.ravel(y_train))
# Salva le metriche di valutazione del modello migliore di Logistic Regression
evaluate_model(X_test, y_test, "Logistic Regression", best_lr_model)
lr_pred=best_lr_model.predict(X_test)
# Calcola le metriche per la classificazione
accuracy_lr = accuracy_score(y_test, lr_pred)

#Naive Bayes
# Importa il modello migliore 
best_nb_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")
# Addestramento dei con tutti i dati di addestramento
best_nb_model.fit(X_train,np.ravel(y_train))
# Salva le metriche di valutazione del miglior modello Naive Bayes Classifier
evaluate_model(X_test, y_test, "Naive Bayes", best_nb_model)
nb_pred=best_nb_model.predict(X_test)
# Calcola le metriche per la classificazione
accuracy_nb = accuracy_score(y_test, nb_pred)

#AdaBoost
#Importa il modello migliore 
best_ab_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")
# Addestramento dei con tutti i dati di addestramento
best_ab_model.fit(X_train,np.ravel(y_train))
# Salva le metriche di valutazione del miglior modello Naive Bayes Classifier
evaluate_model(X_test, y_test, "AdaBoostr", best_ab_model)
ab_pred=best_nb_model.predict(X_test)
# Calcola le metriche per la classificazione
accuracy_ab = accuracy_score(y_test, nb_pred)

# Stampa delle accuratezze
print("Accuratezza XGBOOST:", accuracy_xgboost)
print("Accuratezza SMV:", accuracy_svm)
print("Accuratezza Random Forest:", accuracy_rf)
print("Accuratezza Logistic Regression:", accuracy_lr)
print("Accuratezza Naive Bayes:", accuracy_nb)
print("Accuratezza AdaBoost:", accuracy_ab)

Accuratezza XGBOOST: 0.8425925925925926
Accuratezza SMV: 0.8009259259259259
Accuratezza Random Forest: 0.8379629629629629
Accuratezza Logistic Regression: 0.8148148148148148
Accuratezza Naive Bayes: 0.6342592592592593
Accuratezza AdaBoost: 0.6342592592592593


# **Requirements**

In [10]:
#Comando necessario per ottenere le librerie installate insieme alle loro versioni specifiche.
import subprocess
path = '/content/drive/Shareddrives/biometria_progetto/requirements_SB.txt'
subprocess.run(f'pip freeze > {path}', shell=True, check=True)

CompletedProcess(args='pip freeze > /content/drive/Shareddrives/biometria_progetto/requirements_SB.txt', returncode=0)